# Arbitrage_Sports_Bot.0.1an

### Version #1

In [ ]:
class ArbitrageBot:
    def __init__(self, outcomes):
        self.outcomes = outcomes

    def calculate_opportunity_matrix(self):
        matrix = []
        for i, outcome1 in enumerate(self.outcomes):
            row = []
            for j, outcome2 in enumerate(self.outcomes):
                if i == j:
                    row.append(None)
                else:
                    odds1, odds2 = outcome1['odds'], outcome2['odds']
                    implied_prob1 = 1 / odds1
                    implied_prob2 = 1 / odds2
                    total_prob = implied_prob1 + implied_prob2
                    if total_prob < 1:
                        row.append(None)
                    else:
                        arbitrage_percent = (1 - total_prob) / total_prob
                        row.append({'opportunity': arbitrage_percent, 'bet1': i, 'bet2': j})
            matrix.append(row)
        return matrix

    def calculate_optimal_bets(self, opportunity_matrix, total_stake):
        bets = [0] * len(self.outcomes)
        for row in opportunity_matrix:
            for item in row:
                if item is not None and item['opportunity'] > 0:
                    bet1, bet2 = item['bet1'], item['bet2']
                    odds1, odds2 = self.outcomes[bet1]['odds'], self.outcomes[bet2]['odds']
                    implied_prob1 = 1 / odds1
                    implied_prob2 = 1 / odds2
                    total_prob = implied_prob1 + implied_prob2
                    arb_prob = total_prob - 1
                    arb_stake = (total_stake * arb_prob) / (item['opportunity'] + arb_prob)
                    bets[bet1] += arb_stake / odds1
                    bets[bet2] += arb_stake / odds2
        return bets

    def place_bets(self, bets):
        for i, outcome in enumerate(self.outcomes):
            outcome['bet_amount'] = bets[i]
            outcome['potential_payout'] = bets[i] * outcome['odds']

    def run(self, total_stake):
        opportunity_matrix = self.calculate_opportunity_matrix()
        bets = self.calculate_optimal_bets(opportunity_matrix, total_stake)
        self.place_bets(bets)
        return self.outcomes


### Version #2

In [ ]:
import requests

class ArbitrageBot:
    def __init__(self, bookmaker_odds):
        self.bookmaker_odds = bookmaker_odds
    
    def calculate_arbitrage(self):
        total_prob = 0
        for odds in self.bookmaker_odds:
            total_prob += 1 / odds
        
        if total_prob < 1:
            return {"message": "Arbitrage opportunity found"}
        
        return {"message": "No arbitrage opportunity found"}


url = "https://api.the-odds-api.com/v4/sports/basketball_nba/odds"

params = {
    "apiKey": "0b20b2a9fad61c63a508354884e98914",
    "regions": "us",
    "markets": "spreads",
    "oddsFormat": "american",
    "include": "team"
}

response = requests.get(url, params=params)

if response.status_code == 200:
    data = response.json()[0]
    for game in data:
        home_team = game["teams"][0]["name"]
        away_team = game["teams"][1]["name"]
        home_spread = game["sites"][0]["odds"]["spreads"]["points"][0]
        away_spread = game["sites"][0]["odds"]["spreads"]["points"][1]
        print(f"{away_team} @ {home_team}: {away_spread} - {home_spread}")
else:
    print("Error: Unable to retrieve odds data")

    


### Version #3

In [ ]:
import requests
import json

class ArbitrageBot:
    def __init__(self, api_key):
        self.api_key = '0b20b2a9fad61c63a508354884e98914'
        self.url = "https://api.the-odds-api.com/v3/odds/"
        self.headers = {
            'Content-Type': 'application/json',
            'x-api-key': self.api_key
        }

    def find_arbitrage_opportunities(self, sport):
        # Make a request to the odds-api to get the latest odds for the specified sport
        response = requests.get(self.url + sport, headers=self.headers)
        data = json.loads(response.text)
        
        # Extract the odds and bookmaker information from the response
        bookmakers = data['data'][0]['sites']
        odds = [bookmaker['odds']['h2h'] for bookmaker in bookmakers]
        
        # Calculate the implied probabilities of each outcome
        implied_probs = [1 / odd for odd in odds]
        
        # Calculate the total implied probability for all outcomes
        total_implied_prob = sum(implied_probs)
        
        # Calculate the arbitrage percentage
        arb_percentage = (1 - total_implied_prob) * 100
        
        # Find the bookmaker and outcome that offers the best odds for each outcome
        best_odds = [max(bookmaker['odds']['h2h']) for bookmaker in bookmakers]
        best_bookmakers = [bookmaker['site_key'] for bookmaker in bookmakers]
        
        # Calculate the size of the bets to place on each outcome to guarantee a profit
        bet_sizes = [(1 / odd) * (total_implied_prob - 1) for odd in best_odds]
        
        # Calculate the total amount to bet
        total_bet = sum(bet_sizes)
        
        # Print out the details of the arbitrage opportunity and the recommended bets
        print(f"Arbitrage opportunity found in {sport} with {arb_percentage:.2f}% arbitrage percentage")
        for i in range(len(best_odds)):
            print(f"Bet {bet_sizes[i]:.2f} on {best_bookmakers[i]} for odds of {best_odds[i]}")
        print(f"Total bet size: {total_bet:.2f}")


In [ ]:
bot = ArbitrageBot(api_key='0b20b2a9fad61c63a508354884e98914')
bot.find_arbitrage_opportunities(sport='basketball_nba')

JSONDecodeError: ignored

In [ ]:
import requests


# An api key is emailed to you when you sign up to a plan
# Get a free API key at https://api.the-odds-api.com/
API_KEY = 'YOUR_API_KEY'

SPORT = 'upcoming' # use the sport_key from the /sports endpoint below, or use 'upcoming' to see the next 8 games across all sports

REGIONS = 'us' # uk | us | eu | au. Multiple can be specified if comma delimited

MARKETS = 'h2h,spreads' # h2h | spreads | totals. Multiple can be specified if comma delimited

ODDS_FORMAT = 'decimal' # decimal | american

DATE_FORMAT = 'iso' # iso | unix

# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# First get a list of in-season sports
#   The sport 'key' from the response can be used to get odds in the next request
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

sports_response = requests.get(
    'https://api.the-odds-api.com/v4/sports', 
    params={
        'api_key': API_KEY
    }
)


if sports_response.status_code != 200:
    print(f'Failed to get sports: status_code {sports_response.status_code}, response body {sports_response.text}')

else:
    print('List of in season sports:', sports_response.json())



# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 
#
# Now get a list of live & upcoming games for the sport you want, along with odds for different bookmakers
# This will deduct from the usage quota
# The usage quota cost = [number of markets specified] x [number of regions specified]
# For examples of usage quota costs, see https://the-odds-api.com/liveapi/guides/v4/#usage-quota-costs
#
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # 

odds_response = requests.get(
    f'https://api.the-odds-api.com/v4/sports/{SPORT}/odds',
    params={
        'api_key': API_KEY,
        'regions': REGIONS,
        'markets': MARKETS,
        'oddsFormat': ODDS_FORMAT,
        'dateFormat': DATE_FORMAT,
    }
)

if odds_response.status_code != 200:
    print(f'Failed to get odds: status_code {odds_response.status_code}, response body {odds_response.text}')

else:
    odds_json = odds_response.json()
    print('Number of events:', len(odds_json))
    print(odds_json)

    # Check the usage quota
    print('Remaining requests', odds_response.headers['x-requests-remaining'])
    print('Used requests', odds_response.headers['x-requests-used'])

In [ ]:
bot = ArbitrageBot(api_key='0b20b2a9fad61c63a508354884e98914')
bot.find_arbitrage_opportunities(sport='basketball_nba')